## **1. Import dependencies**

In [213]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import modelbit
mb = modelbit.login()

## **2. Generate some dummy data**

In [194]:
# Generating some random data for a linear regression example
np.random.seed(42)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(np.c_[X, y], columns=['X', 'y'])
df.head()

,X,y
0,0.749080,6.334288
1,1.901429,9.405278
2,1.463988,8.483724
3,1.197317,5.604382
4,0.312037,4.716440


## **3. Perform training splits**

In [195]:
# Split the data: 60% train, 20% validation, 20% test
X_train, X_temp, y_train, y_temp = train_test_split(df[['X']], df['y'], test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## **4. Train model**

In [196]:
# Initialize the linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

LinearRegression()

## **5. Make predictions on validation & test set**

In [197]:
# Make predictions on the validation set
y_val_pred = model.predict(X_val)

# Calculate the mean squared error on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
print(f"Validation MSE: {mse_val}")

Validation MSE: 0.9399074755410866


In [198]:
# Make predictions on the test set
y_test_pred = model.predict(X_test)

# Calculate the mean squared error on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
print(f"Test MSE: {mse_test}")

Test MSE: 0.44585359952764564


## **6. Add model to Modelbit**

In [199]:
mb.add_model("SimpleLinearRegressionModel", model)

Updating registry: 100%|██████████| 1/1 [00:00<00:00]


## **7. Create function to generate predictions**

In [200]:
def regression_predictions(X) -> np.ndarray:
    # Get the model from Modelbit
    model = mb.get_model("SimpleLinearRegressionModel")
    return model.predict(X)

## **8. Deploy function to Modelbit**

In [201]:
scikit_learn_version = sklearn.__version__

mb.deploy(regression_predictions, dataframe_mode = True, example_dataframe = X_train, python_packages=[f"scikit-learn=={scikit_learn_version}"])

## **9. Generate predictions**

In [212]:
single_input = pd.DataFrame([[8], [10]], columns=['X'])
mb.get_inference(workspace="karlmeiermattern", deployment="regression_predictions", data=single_input)

{'data': [[0, 25.64315686857126], [1, 31.000860320727632]]}